In [ ]:
import os
import csv
import numpy as np
import math
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
from skimage import io
from PIL import Image
from tqdm import tqdm
#import argparse

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.utils import save_image

matplotlib.style.use('ggplot')

torch.cuda.empty_cache() 
#import model

In [ ]:
class Encoder(nn.Module):
    def __init__(self, z_dim):
        super(Encoder, self).__init__() 
        self.enc1 = nn.Conv2d(
            in_channels=3, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc2 = nn.Conv2d(
            in_channels=init_kernel, out_channels=init_kernel*2, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc3 = nn.Conv2d(
            in_channels=init_kernel*2, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc4 = nn.Conv2d(
            in_channels=init_kernel*4, out_channels=init_kernel*8, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.enc5 = nn.Conv2d(
            in_channels=init_kernel*8, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        #self.dropout = nn.Dropout2d(p=0.1) # need to call in forward but turn off automatically in eval()
        
        self.mu = nn.Linear(init_kernel*191*191, z_dim)
        self.sigma = nn.Linear(init_kernel*191*191, z_dim)
        
        
    def reparameterize(self, mu, log_var):
        """
        :param mu: mean from the encoder's latent space
        :param log_var: log variance from the encoder's latent space
        """
        std = torch.exp(0.5*log_var) # standard deviation
        eps = torch.randn_like(std) # `randn_like` as we need the same size
        sample = mu + (eps * std) # sampling
        return sample
 
    def forward(self, x):
        
        #print("before anything")
        #print(x.shape)
        x = self.enc1(x)
        x = F.relu(x)
        x = self.enc2(x)
        x = F.relu(x)
        x = self.enc3(x)
        x = F.relu(x)
        #x = self.enc4(x)
        #x = F.relu(x)
        #x = self.enc5(x)
        #x = F.relu(x)
        
        #print("before flatten:")
        #print(x.shape)
        
        x = x.view(x.size(0), -1)
        
        #print("after flatten:")
        #print(x.shape)
        
        # get `mu` and `log_var`
        mu = self.mu(x)
        log_var = self.sigma(x) # fix!!!!name
        
        #return mu, log_var #fix!!! name
        ## get the latent vector through reparameterization
        z = self.reparameterize(mu, log_var)
        
        return z, mu, log_var

class Decoder(nn.Module):
    def __init__(self, z_dim):
        super(Decoder, self).__init__()
        
        self.lin1 = nn.Linear(z_dim, init_kernel*191*191)
        
        self.dec1 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=init_kernel*8, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec2 = nn.ConvTranspose2d(
            in_channels=init_kernel*8, out_channels=init_kernel*4, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec3 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=init_kernel*2, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec4 = nn.ConvTranspose2d(
            in_channels=init_kernel*2, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec5 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=3, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        
    def forward(self, z):
        x = self.lin1(z)
        x=F.relu(x)
        
        x=x.view(-1, init_kernel, 191, 191)
        #print("after unflatten:")
        #print(x.shape)
        
        #x = self.dec1(x)
        #x = F.relu(x)
        #x = self.dec2(x)
        #x = F.relu(x)
        x = self.dec3(x)
        x = F.relu(x)
        x = self.dec4(x)
        x = F.relu(x)
        x = self.dec5(x)
        reconstruction = torch.sigmoid(x)
        
        return reconstruction

class ConvVAE(nn.Module):
    def __init__(self, z_dim):
        super(ConvVAE, self).__init__()
        self.encoder = Encoder(z_dim)
        self.decoder = Decoder(z_dim)
    
    def forward(self,x):
        sample, z_mu, z_logvar = self.encoder(x)
        #z_mu, z_logvar = self.encoder(x)
        
        #std = torch.exp(z_logvar)
        #eps = torch.randn_like(std) # `randn_like` as we need the same size
        #sample = z_mu + (eps * std) # sampling
        
        reconstruction = self.decoder(sample)
        
        return reconstruction, z_mu, z_logvar

Below should be the optimal parameters

In [ ]:
epochs = 25 # 50,100
batch_size = 1 #8,16,32,64?
lr = 0.0001 #0.002, 0.003

kernel_size = 4 #3,4,5,6?
stride = 1 #1,2,3,4?
padding = 0 #0,1,2,3,4?
init_kernel = 8 #4,8,16,32 #initial number of filters

latent_dim = 96

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
class ActiveVisionDataset (Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file, index_col=None)
        self.root_dir = root_dir
        self.transform = transform
        
    def __len__(self):
        return len(self.annotations)
    
    def __getitem__(self,index):
        if type(index) == torch.Tensor:
            index = index.item()
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        shape_label = torch.tensor(int(self.annotations.iloc[index,1]))
        cam_loc = torch.tensor(ast.literal_eval(self.annotations.iloc[index,2]))
        
        if self.transform:
            image = self.transform(image)
        
        return image, shape_label, cam_loc

In [ ]:
dataset = ActiveVisionDataset(csv_file='imgs/rgbCSV.csv', root_dir= 'imgs/rgbImg/', transform = torchvision.transforms.ToTensor())
train_data, val_data = torch.utils.data.random_split(dataset, [2400, 600])
train_loader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_data, batch_size=batch_size, shuffle=True)

In [ ]:
#Which model am i tryna load in huh
PATH = os.path.join(os.getcwd(), "outputs", "batch_sizes8", "batch_sizes8.pth")

ConvVAE = ConvVAE(latent_dim)#.to(device)
ConvVAE.load_state_dict(torch.load(PATH))
ConvVAE.to(device)

In [ ]:
ConvVAE.eval()

In [ ]:
def generate_latent_vectors(model, dataloader):
    model.eval()
    latent = []
    target = []
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, labels = data
            #if torch.cuda.is_available():
            #    data = data.to(device)
            z, mu, logvar = model.encoder(data.cuda())
            latent.extend(mu)
            target.extend(labels)
        return latent, target

In [ ]:

#Maybe don't need this ngl or generate above?
train_latent_var, train_target = generate_latent_vectors(ConvVAE, train_loader)
test_latent_var, test_target = generate_latent_vectors(ConvVAE, val_loader)

# New Stuff

In [ ]:
class CombNet(nn.Module):
    def __init__(self, z_dim):
        super(CombNet, self).__init__()
        
        #Combined fully conntected layer
        #Add this playing around stage later innit
        
        
        #Img decoding
        '''
        self.lin1 = nn.Linear(z_dim, init_kernel*191*191)
        
        self.dec1 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=init_kernel*8, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec2 = nn.ConvTranspose2d(
            in_channels=init_kernel*8, out_channels=init_kernel*4, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        '''
        self.dec3 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=init_kernel*2, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec4 = nn.ConvTranspose2d(
            in_channels=init_kernel*2, out_channels=init_kernel, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        self.dec5 = nn.ConvTranspose2d(
            in_channels=init_kernel, out_channels=3, kernel_size=kernel_size, 
            stride=stride, padding=padding
        )
        
        #Polar Coord decoding
        self.coord1 = nn.linear(z_dim, 50)
        
        self.coord2 = nn.linear(50,10)
        
        self.coord3 = nn.linear(10,3)
        
    
    def forward(self,img,coord):
        
        img, mu, log_var = ConvVAE.encoder(img)
        
        #reshape in some way, maybe expansion?
        x = torch.cat((img, coord), dim=1) #Maybe 0!!!!!!!!!!
        x = self.linear1(z_dim+3, 150)
        x = self.linear2(150,300)
        x = self.linear3(300,z_dim) #Doesn't have to go back to z_dim I was just feeling it 

        ##feed new latent vector into deocders that try to get the original things
        
        #IMG time
        #------------------------------------------------
        img = self.lin1(x)
        img=F.relu(img)
        
        img=img.view(-1, init_kernel, 191, 191)
        #print("after unflatten:")
        #print(x.shape)
        
        #x = self.dec1(x)
        #x = F.relu(x)
        #x = self.dec2(x)
        #x = F.relu(x)
        img = self.dec3(img)
        img = F.relu(img)
        img = self.dec4(img)
        img = F.relu(img)
        img = self.dec5(img)
        reconstruction_img = torch.sigmoid(img)
        #------------------------------------------------
        
        #Coord Time
        #------------------------------------------------
        coord = self.coordlin1(coord)
        coord = F.relu(coord)
        coord = self.coordlin2(coord)
        coord = F.relu(coord)
        coord = self.coordlin3(coord)
        reconstruction_coord = torch.sigmoid(coord)
        
        #------------------------------------------------
        
        
        # decode reshaped vector into full image
        # decode reshaped vector into polar coords
        
        return reconstruction_img, reconstruction_coord
        
#How to do loss?
# % loss per epoch?


# Copied straight from original. Need to mess with them a bit to make work for this!!!

In [ ]:
def final_loss(bce_loss, mu, logvar):
    """
    This function will add the reconstruction loss (BCELoss) and the 
    KL-Divergence.
    KL-Divergence = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    :param bce_loss: recontruction loss
    :param mu: the mean from the latent vector
    :param logvar: log variance from the latent vector
    """
    BCE = bce_loss 
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + KLD

In [ ]:
# To calculate loss:
#Get difference between original and reconstructed img and coord
#Add kldivergence of encoder VAE?

In [ ]:
def fit(model, dataloader):
    model.train()
    running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        #print(data)
        data, _ = data
        if torch.cuda.is_available():
            data = data.to(device)
        optimizer.zero_grad()
        reconstruction, mu, logvar = model(data)
        
        bce_loss = criterion(reconstruction, data)
        loss = final_loss(bce_loss, mu, logvar)
        running_loss += loss.item()
        loss.backward()
        optimizer.step()
    train_loss = running_loss/len(dataloader.dataset)
    return train_loss

In [ ]:
def validate(model, dataloader):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            data, _ = data
            if torch.cuda.is_available():
                data = data.to(device)
            reconstruction, mu, logvar = model(data)
            bce_loss = criterion(reconstruction, data)
            loss = final_loss(bce_loss, mu, logvar)
            running_loss += loss.item()
        
            # save the last batch input and output of every epoch
            if i == int(len(val_data)/dataloader.batch_size) - 1:
                num_rows = 8
                both = torch.cat((data.view(batch_size, 3, 200, 200)[:8], 
                                  reconstruction.view(batch_size, 3, 200, 200)[:8]))
                save_image(both.cpu(), f"outputs/{parameter}{value}/imgs/output{epoch}.png", nrow=num_rows)
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

### New Parameters for NEW model!

In [ ]:
# leanring parameters

#Number of conv layers: 3,4,5

epochs = 25 # 50,100
batch_size = 8 #8,16,32,64?
lr = 0.0001 #0.002, 0.003

kernel_size = 4 #3,4,5,6?
stride = 1 #1,2,3,4?
padding = 0 #0,1,2,3,4?
init_kernel = 8 #4,8,16,32 #initial number of filters

latent_dim = 96

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

#Add remove layers ***make priority!!!!
#Optimizer (setup just a few cells below)

#Do dropout (across multiple layers, with multiple p values?)
#Do pooling? Max pool or average pool?
#Do activation function #There's a lot
#Batch normalization #Looked it up